<a href="https://colab.research.google.com/github/Cha-iysong/Taxi-demand-prediction-in-New-York-City/blob/master/1117(%ED%83%9D%EC%8B%9C2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Importing Libraries
# pip3 install graphviz
#pip3 install dask
#pip3 install toolz
#pip3 install cloudpickle
# https://www.youtube.com/watch?v=ieW3G7ZzRZ0
# https://github.com/dask/dask-tutorial
# please do go through this python notebook: https://github.com/dask/dask-tutorial/blob/master/07_dataframe.ipynb
import dask.dataframe as dd#similar to pandas

import pandas as pd#pandas to create small dataframes 

# pip3 install folium
import folium #open street map

# unix time: https://www.unixtimestamp.com/
import datetime #Convert to unix time

import time #Convert to unix time

# if numpy is not installed already : pip3 install numpy
import numpy as np#Do aritmetic operations on arrays

# matplotlib: used to plot graphs
import matplotlib
# matplotlib.use('nbagg') : matplotlib uses this protocall which makes plots more user intractive like zoom in and zoom out
matplotlib.use('nbagg')
import matplotlib.pylab as plt
import seaborn as sns#Plots
from matplotlib import rcParams#Size of plots  

# this lib is used while we calculate the stight line distance between two (lat,lon) pairs in miles
!pip3 install gpxpy
import gpxpy.geo #Get the haversine distance

from sklearn.cluster import MiniBatchKMeans, KMeans#Clustering
import math
import pickle
import os

# download migwin: https://mingw-w64.org/doku.php/download/mingw-builds
# install it in your system and keep the path, migw_path ='installed path'
mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-5.3.0-posix-seh-rt_v4-rev0\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

# to install xgboost: pip3 install xgboost
# if it didnt happen check install_xgboost.JPG
import xgboost as xgb

# to install sklearn: pip install -U scikit-learn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 111 kB 24.0 MB/s 
  Created wheel for gpxpy: filename=gpxpy-1.5.0-py3-none-any.whl size=42899 sha256=f86279f15138d5a6958c16890bf4012d7d023e690ac1e7095d8516ce8b56c124
  Stored in directory: /root/.cache/pip/wheels/78/49/b6/d1851fcb4ee10fa784325d7fcf6c97789157c706ab3a71e2b6
Successfully built gpxpy


## Data Collection

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
filename_gzip = '/content/drive/MyDrive/2016_01_Yellow_Taxi_Trip_Data.csv'

In [12]:
month_csv = pd.read_csv(filename_gzip, nrows=1000000, error_bad_lines=False)

In [33]:
month_csv.head(100)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,...,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PULocationID,DOLocationID
0,2,01/01/2016 12:00:00 AM,01/01/2016 12:18:30 AM,2,5.52,-73.980118,40.743050,1,N,-73.913490,...,2,19.0,0.5,0.5,0.00,0.0,0.3,20.30,NaN,NaN
1,2,01/01/2016 12:00:00 AM,01/01/2016 12:00:00 AM,3,1.76,-73.960625,40.781330,1,N,-73.977264,...,2,8.0,0.0,0.5,0.00,0.0,0.3,8.80,NaN,NaN
2,2,01/01/2016 12:00:00 AM,01/01/2016 12:00:00 AM,1,10.54,-73.984550,40.679565,1,N,-73.950272,...,1,33.0,0.5,0.5,0.00,0.0,0.3,34.30,NaN,NaN
3,2,01/01/2016 12:00:00 AM,01/01/2016 12:26:45 AM,2,7.45,-73.994057,40.719990,1,N,-73.966362,...,2,26.0,0.5,0.5,0.00,0.0,0.3,27.30,NaN,NaN
4,2,01/01/2016 12:00:00 AM,01/01/2016 12:00:00 AM,1,4.75,-73.993469,40.718990,1,N,-73.962242,...,2,16.5,0.0,0.5,0.00,0.0,0.3,17.30,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,01/01/2016 12:01:08 AM,01/01/2016 12:23:22 AM,1,5.00,-74.002266,40.730370,1,N,-73.951683,...,1,19.5,0.5,0.5,4.50,0.0,0.3,25.30,NaN,NaN
96,2,01/01/2016 12:01:09 AM,01/01/2016 12:05:23 AM,1,0.58,-73.998077,40.737968,1,N,-74.004128,...,1,5.0,0.5,0.5,1.26,0.0,0.3,7.56,NaN,NaN
97,1,01/01/2016 12:01:09 AM,01/01/2016 12:06:37 AM,2,1.60,-73.980827,40.779640,1,N,-73.971413,...,1,6.5,0.5,0.5,1.55,0.0,0.3,9.35,NaN,NaN
98,1,01/01/2016 12:01:10 AM,01/01/2016 12:36:55 AM,2,6.00,-73.999443,40.729836,1,N,-73.964180,...,1,25.5,0.5,0.5,5.36,0.0,0.3,32.16,NaN,NaN


In [32]:
print(month_csv.columns)

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RatecodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'PULocationID',
       'DOLocationID'],
      dtype='object')


## Data Cleaning

In [31]:
month = dd.from_pandas(month_csv, npartitions=1)

## X & Y Coordinates

In [36]:
# Plotting pockup cordinates which are outside the bounding box of New-York
# we will collect all the points outside the bounding box of newyork city to outlier_locations
outlier_locations = month[(month.pickup_longitude <= -74.15) | (month.pickup_latitude <= 40.5774)| \
                          (month.pickup_longitude >= 73.7004) | (month.pickup_latitude >= 40.9176)]

# creating a map with the a base location
# read more about the folium here: http://folium.readthedocs.io/en/latest/quickstart.html

# note: you dont need to remeber any of these, you dont need indeepth knowledge on these maps and plots

map_osm = folium.Map(location=[40.734695, -73.990372], tiles='Stamen Toner')

# we will spot only first 100 outliers on the map, plotting all the outliers will take more time
sample_locations = outlier_locations.head(10000)
for i,j in sample_locations.iterrows(): 
    if int(j['pickup_latitude']) != 0:
        folium.Marker(list((j['pickup_latitude'],j['pickup_longitude']))).add_to(map_osm)
map_osm